# Deep Learning for Sentiment Analysis

This notebook aims to provide an example of how a Recurrent Neural Network (RNN) using the Long Short Term Memory (LSTM) architecture can be implemented using Keras(https://keras.io/). Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. 

In this notebook, we have experimented with LSTM to perform sentiment analysis on movie reviews from the Large Movie Review Dataset(http://ai.stanford.edu/~amaas/data/sentiment/), better known as the IMDB dataset.

In this task, given a movie review, the model attempts to predict whether it is positive or negative. This is a binary classification task.

In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


## Data

As previously mentioned, we shall train a LSTM recurrent neural network on the Large Movie Review Dataset dataset.

The Large Movie Review Dataset (often referred to as the IMDB dataset) contains 25,000 highly-polar movie reviews (good or bad) for training and the same amount again for testing. The problem is to determine whether a given movie review has a positive or negative sentiment.

The data was collected by Stanford researchers and was used in a 2011 paper where a split of 50-50 of the data was used for training and test. **An accuracy of 88.89% was achieved.**

### Keras advantages:
As stated earlier, Keras was built with a focus on fast experimentation and prototyping. Hence,Keras provides access to the IMDB dataset built-in! 

The **imdb.load_data()** function allows you to load the dataset in a format that is ready for use in neural network and deep learning models.

The words have been replaced by integers that indicate the ordered frequency of each word in the dataset. The sentences in each review are therefore comprised of a sequence of integers.

In [2]:
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

## Word Embeddings

The vocabulary of words in all the reviews is very large. Mere one-hot encoding of individual words will lead to an extremely sparse dataset.

Hence we will use Word Embeddings, a technique where words are encoded as real-valued vectors in a high dimensional space, where the similarity between words in terms of meaning translates to closeness in the vector space. This reduces sparsity of the data, as well as gives more meaning to each word in this embedded space, rather than just present or not. For more information on word embeddings, check out my experiments with Word Embeddings on Harry Potter and Game of Thrones [here](https://github.com/darshanbagul/Word_Embeddings)!

In this notebook we won't have to use Gensim or create an embedding network from scratch. This is another advantage of using Keras. We just include another layer after the input in our model for generating embeddings of the input word! Keras provides a convenient way to convert positive integer representations of words into a word embedding by an **Embedding layer.**

We will map each word onto a **32 length real valued vector**. We will also limit the total number of words that we are interested in modeling to the **5000 most frequent words, and zero out the rest.** Finally, the sequence length (number of words) in each review varies, so we will **constrain each review to be 500 words,** truncating long reviews and pad the shorter reviews with zero values. (Another alternative for this could be experimenting with a Dynamic RNN, but that is something I shall experiment with later.)

In [3]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

## Models

Now that we have preprocessed the dataset and split into train and test, let us begin exploring with different model architectures. The models that we are going to implement are listed below with increasing complexity:

    1. A Simple LSTM Network
    2. LSTM Network with Dropout regularization
    3. CNN + LSTM with Dropout regularization

Let us dive right in with each model!

### 1. A Simple LSTM Network

We can quickly develop a small LSTM for the IMDB sentiment classification and achieve good accuracy as we will see below. The model architecture is printed after defining the model:

In [4]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
model.fit(X_train, y_train, epochs=4, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
25000/25000 [==============================] - 988s - loss: 0.5734 - acc: 0.6856   
Epoch 2/4
25000/25000 [==============================] - 991s - loss: 0.3090 - acc: 0.8736   
Epoch 3/4
25000/25000 [==============================] - 1325s - loss: 0.2415 - acc: 0.9054  
Epoch 4/4
25000/25000 [==============================] - 1662s - loss: 0.2174 - acc: 0.9150  
Accuracy: 85.72%


### 2. LSTM Network With Dropout

RNNs like LSTM generally suffer from overfitting the data. Dropout is a technique of countering overfitting by improving the ability of model to generalise better. This is done by setting random activations of hidden layers to zero. Through this randomness, the final classification layer always receives a different transformed input for a corresponding output, thus preventing overfitting and improving model's ability to generalize on unseen data. 

Dropout can be applied between layers using the Dropout Keras layer. We can do this easily by 
    1. Adding new Dropout layers between the Embedding and LSTM layers and the LSTM and Dense output layers, or 
    2. As parameters on the LSTM layer; the 'dropout' for configuring the input dropout and 'recurrent_dropout' for configuring the recurrent dropout.
    
Here we will be using the latter option, just for the sake of a compact solution.

In [7]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

The model architecture looks as shown below:

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(X_train, y_train, epochs=3, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
25000/25000 [==============================] - 1675s - loss: 0.5034 - acc: 0.7552  
Epoch 2/3
25000/25000 [==============================] - 1707s - loss: 0.3894 - acc: 0.8316  
Epoch 3/3
25000/25000 [==============================] - 4068s - loss: 0.3404 - acc: 0.8586  
Accuracy: 85.67%


As we can see, the model accuracy took a hit, but the model generalizes better. This model is more robust to noise and changes in data representations, as compared to the previous model. 

Dropout is a powerful technique for combating overfitting in LSTM models!

Let us now try and combine Convolutional Neural Networks along with LSTM to check if we are able to increase the model accuracy.

### 3. LSTM and Convolutional Neural Network with dropout

Convolutional neural networks (CNNs) generally excel at learning the spatial structure in input data. Hence they are widely used on data which comprise of highly correlated spatial structures. For example - Images! Images are unstructured data points, where groups of pixels represent a particular structure. Presence or absence of such structures, helps us classify images into particular category.

The IMDB review data does have a one-dimensional spatial structure in the sequence of words in reviews and the CNN may be able to pick out invariant features for good and bad sentiment. This learned spatial features may then be learned as sequences by an LSTM layer.

Adding a 1-D Convolutional layer followed by Max pooling is elementary in Keras. 

Here we will add a 1-D Conv layer and max pooling layer after the Embedding layer which then feed the consolidated features to the LSTM. We use a set of 32 feature maps (convolutional filters) with a size of 3x3. The pooling layer can use the standard length of 2 to halve the feature map size.

In [29]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Let us visualize the model architecture below:

In [11]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 250, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None

In [30]:
model.fit(X_train, y_train, epochs=4, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/4
25000/25000 [==============================] - 530s - loss: 0.4880 - acc: 0.7500   
Epoch 2/4
25000/25000 [==============================] - 501s - loss: 0.3197 - acc: 0.8676   
Epoch 3/4
25000/25000 [==============================] - 544s - loss: 0.2695 - acc: 0.8927   
Epoch 4/4
25000/25000 [==============================] - 557s - loss: 0.2383 - acc: 0.9053   
Accuracy: 88.10%


In [22]:
model.fit(X_train, y_train, epochs=1, batch_size=64)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/1
25000/25000 [==============================] - 561s - loss: 0.1997 - acc: 0.9222   
Accuracy: 88.18%


## Results

As we can see, with a mixture of a **Convolutional Neural network + LSTM**, we are able to achieve an accuracy as high as the state-of-art accuracy for this dataset. As stated earlier, the Stanford researchers were able to achieve an accuracy of 88.89%, and **we have been able to reach 88.18%** (I even achieved **88.41%** with a different seed earlier!)

Some reflections:
    1. CNN and max pooling layers after the Embedding layer are able to pick out invariant features for good and bad sentiment. These learned spatial features are then learned as sequences by an LSTM layer.
    2. We have less weights and faster training time!

## Summary

In this notebook we implemented LSTM network models for sequence classification problem, specifically sentiment classification of movie reviews from **Large Movie Review Dataset dataset.**

    1. Implemented a simple single layer LSTM model for the IMDB movie review sentiment classification problem.
    2. Extended LSTM model with LSTM-specific dropout to combat overfitting.
    3. Combined the spatial structure learning properties of a Convolutional Neural Network (CNN) with the sequence learning of an LSTM.